# Extraction - Basic
* 고속도로 연결로 접속부 :: 분류부 영향권(2, 3번 영상) 분석

# Imports

In [1]:
import numpy as np
import pandas as pd

import os
from tqdm import tqdm

# Extraction by LOS
* `01_density_by_frame` 폴더의 내용을 바탕으로 LOS별 추출할 프레임을 지정
* **다음을 먼저 작성해야 합니다** : `LOS_frame_list.csv`

## Extracting by LOS
* 각 대상지(영상)별 LOS별로 분석할 프레임만 필터링(20초 이상 = 600개 프레임 이상, LOS별)
* 해당 프레임에 속한 Vehicle ID를 추적
* * `LOS_frame_list`에 명시된 LOS별 프레임들을 추출하여 .csv 파일로 저장

### Load Frame Info

In [2]:
frame_rate = 30 # 프레임율 설정

In [3]:
frm_info_path = 'LOS_frame_list_B_affected.csv'

In [4]:
frm_info = pd.read_csv(frm_info_path)
#frm_info.reset_index(inplace = True)

In [5]:
frm_info['tmsec'] = (frm_info['end_at'] - frm_info['start_at'])/frame_rate # 파생변수 생성: 프레임ID 차이에 따른 시간차

In [6]:
frm_info.head()

,num,LOS,CNT,start_at,end_at,tmsec
0,02_3,B,1,507,1266,25.3
1,02_3,C,1,1266,1560,9.8
2,02_3,A,1,1650,1830,6.0
3,02_3,B,2,1830,2244,13.8
4,02_3,C,2,2244,2493,8.3


### calculate tmsec
* frm_info로부터 각 대상지, LOS별 tmsec 총 관측시간(tmsec) 산정
* 이를 `LOS_tmsec.csv` 로 저장

In [7]:
tmsec_info = pd.pivot_table(frm_info, index = ['num', 'LOS'], values = 'tmsec', aggfunc = 'sum').reset_index()

tmsec_info.head()

,num,LOS,tmsec
0,02_3,A,315.800000
1,02_3,B,254.400000
2,02_3,C,67.200000
3,02_4,A,310.833333
4,02_4,B,439.966667


In [8]:
tmsec_path = 'LOS_tmsec_B_affected.csv'

In [9]:
tmsec_info.to_csv(tmsec_path, encoding = 'utf-8')

### Load Datasets

In [10]:
folder_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/17_드론궤적분석자료_2점분합류_금호JC_부산방향/B_affected'

In [11]:
folder_name = '00_dataset_filtered' # 각 .csv 파일이 있는 폴더경로

file_dir = os.path.join(folder_dir, folder_name)

In [12]:
file_list = os.listdir(file_dir)
file_list[0:3]

['02_3.csv', '02_4.csv']

In [13]:
df_list = [] # 데이터프레임 리스트
num_list = [] # 데이터프레임 파일명, 즉 넘버링 리스트

for file in file_list:
    num = file[0:4]
    file_path = os.path.join(file_dir, file)
            
    if os.path.isfile(file_path): # 해당 file_path에 파일이 존재할 경우
        globals()[f'target{num}'] = pd.read_csv(file_path, encoding = 'cp949') # .csv 파일을 불러와 인스턴스 만들기
        df_list.append(globals()[f'target{num}']) 
        num_list.append(num)

In [15]:
#target01_2.head(3)

### Extract & Save

In [16]:
folder_name = '01_density_by_target_LOS'

save_extract_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_extract_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [17]:
for i in range(len(frm_info)): # frm_info에 따라서
    
    num = frm_info['num'][i] # 대상지 번호. tg_spot. 01_1 등으로 표현
    LOS = frm_info['LOS'][i] # 대상지 촬영지점의 해당 LOS
    CNT = frm_info['CNT'][i] # 몇번쨰 LOS인지 표시
    
    frame_start = frm_info['start_at'][i] # 시작 프레임
    frame_end = frm_info['end_at'][i] # 끝 프레임
    
    tmsec = frm_info['tmsec'][i] # 총 시간
    
    df = globals()[f'target{num}']
    
    # LOS별로 프레임을 추출하여 각 대상지별, LOS별 파일로 저장
    df_extract = df[(df['Frame ID'] >= frame_start) & (df['Frame ID'] <= frame_end)] # 프레임별로 필터링한 df
    
    save_extract_name = f'{num}_{LOS}_{CNT}.csv'
    save_extract_path = os.path.join(save_extract_dir, save_extract_name)
    
    df_extract.to_csv(save_extract_path, encoding = 'cp949')

## Concat and Save
* `01_denstiy_by_target_LOS` 에 있는 .csv 파일들을 대상지별, LOS별로 병합
* `01_denstiy_by_target_LOS_concated`에 저장하기

### Load Dataset
* 대상지 및 LOS 조각별로 쪼개진 데이터들을 불러오기

In [18]:
folder_name = '01_density_by_target_LOS'

file_dir = os.path.join(folder_dir, folder_name)

In [19]:
file_list = os.listdir(file_dir)
file_list[0:3]

['02_3_A_1.csv', '02_3_A_10.csv', '02_3_A_11.csv']

In [20]:
df_list = []
num_list = []
LOS_list = []
fraction_list = []


for file in file_list:
    num = file[0:4] # 파일의 대상지 번호. tg_spot
    LOS = file[5]
    fraction = file[7:-4] # 파일의 조각 번호
    
    file_path = os.path.join(file_dir, file)
    globals()[f'target{num}_{LOS}_{fraction}'] = pd.read_csv(file_path, encoding = 'cp949')
    
    df_list.append(globals()[f'target{num}_{LOS}_{fraction}'])
    
    if num not in num_list:
        num_list.append(num)
    
    if LOS not in LOS_list:
        LOS_list.append(LOS)
        
    if fraction not in fraction_list:
        fraction_list.append(fraction)

In [22]:
#target01_2_A_1.head(3)

### RUN and Save

In [23]:
folder_name = '01_denstiy_by_target_LOS_concated'

save_dir = os.path.join(folder_dir, folder_name)

os.makedirs(save_dir, exist_ok = True) # 해당 경로가 없을 시 폴더 생성, 존재할 경우 건너뛰기

In [24]:
fraction_list

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29']

In [25]:
for num in num_list:
    for LOS in LOS_list:
        LOS_df = pd.DataFrame() # 대상지별, LOS별 빈 데이터프레임 생성
        
        for fraction in fraction_list:
            
            if f'target{num}_{LOS}_{fraction}' in globals(): # 해당 이름의 변수가 전역변수(globals()) 내에 존재할 시
                frac_df = globals()[f'target{num}_{LOS}_{fraction}']
                LOS_df = pd.concat([LOS_df, frac_df])
            else:
                pass
        
        LOS_save_name = num + '_' + LOS + '.csv'
        LOS_save_path = os.path.join(save_dir, LOS_save_name)
        
        LOS_df.to_csv(LOS_save_path, encoding = 'cp949') # LOS별로 병합된 파일을 저장